In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval

# Task 1 Класичний нейрон з довільною кількістю входів

### Режим навчання

In [2]:
frame = pd.read_csv('input_1.csv', sep=',', index_col=0)
data = frame.to_dict()

In [3]:
x = np.array(literal_eval(data['Data']['X']), dtype=float)
y_expected = float(data['Data']['Y_expected'])
W = np.array(literal_eval(data['Data']['W']), dtype=float)
delta_max = float(data['Data']['delta_max'])

In [4]:
class classifier():
    
    def fit(self, x, y_expected, W, delta_max):

        count = 0
        while True:
            count += 1
            X = np.dot(W.T, x)
            y = 1/(1+np.exp(-X))
            delta = np.abs((y_expected - y)/y_expected)
            if delta < delta_max:
                break
            else:
                delta_y = y*(1-y)*(y_expected - y)
                delta_w = delta_y * x
                W += delta_w
        
        return count, W, y
    
    def predict(self, x, W):
        X = np.dot(W.T, x)
        y = 1/(1+np.exp(-X))
        return y

In [5]:
count, W, y = classifier().fit(x, y_expected, W, delta_max)
accuracy = 100 - np.abs((y_expected - y)/y_expected)*100

In [6]:
frame = pd.DataFrame({'Data': [count, np.array2string(W, precision=7, separator=', '), y_expected, y, str(accuracy)+" %"]},
                     index=['Кількість ітерацій', 'W', 'Y очікуване', 'Y реальне', 'Accuracy'])

In [7]:
frame.to_csv('output_1.csv', sep=',', header=True, index=True)

In [8]:
frame.style.set_properties(subset=['Data'], **{'width': '450px'})

,Data
Кількість ітерацій,237
W,"[ 0.1272668, -0.0154664, 0.0154338, 0.1909002]"
Y очікуване,0.7
Y реальне,0.705454
Accuracy,99.2208976334287 %


### Режим розпізнавання

In [9]:
y = classifier().predict([6, 1, 4, 8], W)
y_expected = 0.7
accuracy = 100 - np.abs((y_expected - y)/y_expected)*100
print("Y реальне: "+str(y))
print("Y очікуване: "+str(y_expected))
print(str(accuracy)+' %')

Y реальне: 0.9119010634317091
Y очікуване: 0.7
69.72841950975584 %


# Task 2 ДШП

### Режим навчання

In [10]:
frame = pd.read_csv('input_2.csv', sep=',', index_col=0)
data = frame.to_dict()

In [11]:
x = np.array(literal_eval(data['Data']['X']), dtype=float)
y_expected = np.array(literal_eval(data['Data']['Y_expected']))
L = int(data['Data']['L'])
W = {}
for i in range(1, L+1):
    W['W'+str(i)] = np.array(literal_eval(data['Data']['W'+str(i)]), dtype=float)
delta_max = float(data['Data']['delta_max'])

In [12]:
class NN_classifier():
    
    def linear_forward(self, A, W):
        return np.dot(W, A)

    def linear_activation_forward(self, A_prev, W, activation):
        Z = self.linear_forward(A_prev, W)
        if activation == 'sigmoid':
            A = 1/(1+np.exp(-Z))
        elif activation == 'relu':
            A = np.fmax(0, Z)
        elif activation == 'tanh':
            A = (np.exp(2*Z)-1)/(np.exp(2*Z)+1)
        return A
    
    def linear_activation_backward(self, y, activation):
        if activation == 'sigmoid':
            return y*(1-y)
        elif activation == 'relu':
            return np.array([0 if j==0 else 1 for j in y[0]])
        elif activation == 'tanh':
            return 1-y**2
    
    def forward_propagation(self, y, L, W, activation):
        
        for i in range(L-1):
            y_prev = y[str(i)]
            y[str(i+1)] = self.linear_activation_forward(y_prev, W['W'+str(i+1)], activation)
        y_prev = y[str(L-1)]
        y[str(L)] = self.linear_activation_forward(y_prev, W['W'+str(L)], 'sigmoid')
        
        return y
    
    def backward_propagation(self, y, L, W, delta_max, activation):
        delta_y = {}
        delta_w = {}
        delta_y[str(L-1)] = self.linear_activation_backward(y[str(L)], 'sigmoid')*(y[str(L+1)]-y[str(L)])
        
        assert(delta_y[str(L-1)].shape == y[str(L)].shape)
        
        delta_w[str(L-1)] = np.dot(delta_y[str(L-1)], y[str(L-1)].T)
        W['W'+str(L)] += delta_w[str(L-1)]
        
        assert(W['W'+str(L)].shape == delta_w[str(L-1)].shape)
        
        for i in range(L-1, 0, -1):
            delta_y[str(i-1)] = self.linear_activation_backward(y[str(i)], activation)* \
                np.dot(W['W'+str(i+1)].T, delta_y[str(i)])
            
            assert(delta_y[str(i-1)].shape == y[str(i)].shape)
            
            delta_w[str(i-1)] = np.dot(delta_y[str(i-1)], y[str(i-1)].T)
            W['W'+str(i)] += delta_w[str(i-1)]
            
            assert(W['W'+str(i)].shape == delta_w[str(i-1)].shape)
            
        return W, y[str(L)]
    
    def fit(self, x, y_expected, L, W, delta_max, activation):
        
        y = {'0': x,
             str(L+1): y_expected}
        count = 0
        while True:
            count += 1
            y = self.forward_propagation(y, L, W, activation)
            delta = np.linalg.norm((y[str(L+1)] - y[str(L)])/y[str(L+1)])
            if delta < delta_max:
                break
            else:
                W, y_real = self.backward_propagation(y, L, W, delta_max, activation)
        
        return count, W, y_real
    
    def predict(self, x, W, L, activation):
        
        y = {'0': x}
        y = self.forward_propagation(y, L, W, activation)
            
        return y[str(L)]

In [13]:
count, W, y_real = NN_classifier().fit(x, y_expected, L, W, delta_max, 'relu')

In [14]:
frame = pd.DataFrame({'Data': [count]}, index=['Кількість ітерацій'])
for i in range(1, L+1):
    frame = frame.append(pd.DataFrame({'Data': [np.array2string(W['W'+str(i)], precision=7, separator=', ')]},
                              index=['W'+str(i)]))
frame = frame.append(pd.DataFrame({'Data': [y_expected, y_real]}, index=['Y очікуване', 'Y реальне']))

In [15]:
frame.to_csv('output_2.csv', sep=',', header=True, index=True)

In [16]:
frame.style.set_properties(subset=['Data'], **{'width': '450px'})

,Data
Кількість ітерацій,6
W1,"[[ 0.3301722, 0.2803444, 0.22431 ], [ 0.134881 , -0.360238 , -0.1152142], [ 0.1551074, 0.2402148, -0.1868066], [ 0.0293442, -0.1613116, 0.0288196]]"
W2,"[[ 0.2660791, 0.21 , 0.2456828, -0.1330033], [-0.5366457, 0.32 , 0.0175792, 0.1467739]]"
Y очікуване,[[0.7] [0.2]]
Y реальне,[[0.69400939] [0.20370568]]


### Режим розпізнавання

In [17]:
y_real = NN_classifier().predict([[8], [-2.7], [0.2]], W, L, 'relu')
y_expected = np.array([[0.7], [0.2]])
print("Y реальне: "+str(y_real))
print("Y очікуване: "+str(y_expected))

Y реальне: [[0.72825376]
 [0.43113895]]
Y очікуване: [[0.7]
 [0.2]]
